In [4]:
!pip install tensorflow opencv-python mediapipe sklearn matplotlib

Defaulting to user installation because normal site-packages is not writeable


In [5]:
!pip install sklearn

Defaulting to user installation because normal site-packages is not writeable


In [6]:
!pip install tensorflow-gpu

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  ERROR: Error [WinError 225] Operation did not complete successfully because the file contains a virus or potentially unwanted software while executing command python setup.py egg_info
ERROR: Could not install packages due to an OSError: [WinError 225] Operation did not complete successfully because the file contains a virus or potentially unwanted software



In [7]:
!pip install mediapipe

Defaulting to user installation because normal site-packages is not writeable


In [8]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable


In [9]:
!pip install sklearn matplotlib

Defaulting to user installation because normal site-packages is not writeable


In [16]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [17]:
mp_holistic = mp.solutions.holistic    # Holistic Model
mp_drawing = mp.solutions.drawing_utils    # Drawing Utilities

In [18]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [19]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [20]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
                             )
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2)
                             )
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
                             )
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245, 137, 66), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(245, 66, 250), thickness=2, circle_radius=2)
                             )

In [15]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)
        
        draw_styled_landmarks(image, results)

        cv2.imshow('OpenCV Feed', image)
        if cv2.waitKey(10) & 0xFF== ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [6]:
len(results.left_hand_landmarks.landmark)

NameError: name 'results' is not defined

In [21]:
draw_landmarks(frame, results)

In [ ]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

In [7]:
results

NameError: name 'results' is not defined

In [8]:
all_landmarks = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    all_landmarks.append(test)

NameError: name 'results' is not defined

In [22]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    
    return np.concatenate([pose, face, lh, rh])

In [10]:
extract_keypoints(results).shape

NameError: name 'results' is not defined

In [23]:
DATA_PATH = os.path.join("MP_Data")
actions = np.array(['hello', 'thanks', 'iloveyou', 'what', 'bad'])
no_sequences = 30
sequence_length = 30

In [24]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [54]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):
                ret, frame = cap.read()

                image, results = mediapipe_detection(frame, holistic)

                draw_styled_landmarks(image, results)
                
                if frame_num == 0:
                    cv2.putText(image, 'Starting Collection', (120, 200),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting Frames for {} Video Number {}'.format(action, sequence), (15, 12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image, 'Collecting frames for {} Video NUmber {}'.format(action, sequence), (15, 12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                
                cv2.imshow('OpenCV Feed', image)
                
                if cv2.waitKey(10) & 0xFF== ord('q'):
                    break
cap.release()
cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [13]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [25]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [26]:
label_map = {label:num for num, label in enumerate(actions)}

In [27]:
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2, 'what': 3, 'bad': 4}

In [28]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [29]:
X = np.array(sequences)

In [30]:
y = to_categorical(labels).astype(int)

In [31]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [32]:
x_train

array([[[ 0.62872851,  0.48193341, -0.73823678, ...,  0.46854499,
          0.80924106, -0.06774545],
        [ 0.62882149,  0.48195583, -0.71378702, ...,  0.47970781,
          0.82463014, -0.06230075],
        [ 0.62918818,  0.48195547, -0.72312886, ...,  0.47876698,
          0.81759489, -0.06600452],
        ...,
        [ 0.62417662,  0.48306602, -0.68499726, ...,  0.47863635,
          0.82331115, -0.06326684],
        [ 0.62393868,  0.48338005, -0.68481386, ...,  0.47770819,
          0.82320738, -0.06063944],
        [ 0.62375647,  0.48365882, -0.68474114, ...,  0.47710711,
          0.82189167, -0.0614552 ]],

       [[ 0.56632102,  0.50476819, -0.55763918, ...,  0.6065765 ,
          0.58916783, -0.03102761],
        [ 0.56686276,  0.50443375, -0.55912733, ...,  0.60815865,
          0.58813787, -0.0347853 ],
        [ 0.56676954,  0.50290525, -0.58560669, ...,  0.60691714,
          0.58665973, -0.03803128],
        ...,
        [ 0.56799555,  0.50433791, -0.54840088, ...,  

In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [34]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [35]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [36]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [37]:
model.fit(x_train, y_train, epochs=1000, callbacks=[tb_callback])

Epoch 1/1000
5/5 [==============================] - 8s 64ms/step - loss: 6.5926 - categorical_accuracy: 0.2113
Epoch 2/1000
5/5 [==============================] - 0s 69ms/step - loss: 4.5863 - categorical_accuracy: 0.1831
Epoch 3/1000
5/5 [==============================] - 0s 70ms/step - loss: 1.7042 - categorical_accuracy: 0.2324
Epoch 4/1000
5/5 [==============================] - 0s 70ms/step - loss: 1.6803 - categorical_accuracy: 0.2324
Epoch 5/1000
5/5 [==============================] - 0s 70ms/step - loss: 2.9532 - categorical_accuracy: 0.2324
Epoch 6/1000
5/5 [==============================] - 0s 73ms/step - loss: 3.2829 - categorical_accuracy: 0.1620
Epoch 7/1000
5/5 [==============================] - 0s 76ms/step - loss: 1.6813 - categorical_accuracy: 0.1338
Epoch 8/1000
5/5 [==============================] - 0s 70ms/step - loss: 1.7223 - categorical_accuracy: 0.1479
Epoch 9/1000
5/5 [==============================] - 0s 68ms/step - loss: 1.6698 - categorical_accuracy: 0.1620
E

5/5 [==============================] - 0s 74ms/step - loss: 1.2692 - categorical_accuracy: 0.4296
Epoch 75/1000
5/5 [==============================] - 0s 72ms/step - loss: 1.3403 - categorical_accuracy: 0.4225
Epoch 76/1000
5/5 [==============================] - 0s 74ms/step - loss: 1.1841 - categorical_accuracy: 0.5493
Epoch 77/1000
5/5 [==============================] - 0s 72ms/step - loss: 1.5176 - categorical_accuracy: 0.3944
Epoch 78/1000
5/5 [==============================] - 0s 76ms/step - loss: 1.7281 - categorical_accuracy: 0.2042
Epoch 79/1000
5/5 [==============================] - 0s 77ms/step - loss: 1.6005 - categorical_accuracy: 0.2183
Epoch 80/1000
5/5 [==============================] - 0s 75ms/step - loss: 1.5905 - categorical_accuracy: 0.1972
Epoch 81/1000
5/5 [==============================] - 0s 72ms/step - loss: 1.5546 - categorical_accuracy: 0.2113
Epoch 82/1000
5/5 [==============================] - 0s 72ms/step - loss: 1.5790 - categorical_accuracy: 0.2887
Epoch 

5/5 [==============================] - 0s 76ms/step - loss: 1.0325 - categorical_accuracy: 0.5141
Epoch 147/1000
5/5 [==============================] - 0s 75ms/step - loss: 1.1004 - categorical_accuracy: 0.5070
Epoch 148/1000
5/5 [==============================] - 0s 74ms/step - loss: 1.0027 - categorical_accuracy: 0.5211
Epoch 149/1000
5/5 [==============================] - 0s 74ms/step - loss: 0.9901 - categorical_accuracy: 0.5423
Epoch 150/1000
5/5 [==============================] - 0s 76ms/step - loss: 0.9762 - categorical_accuracy: 0.5070
Epoch 151/1000
5/5 [==============================] - 0s 75ms/step - loss: 0.9262 - categorical_accuracy: 0.5282
Epoch 152/1000
5/5 [==============================] - 0s 74ms/step - loss: 1.3037 - categorical_accuracy: 0.4085
Epoch 153/1000
5/5 [==============================] - 0s 74ms/step - loss: 1.5120 - categorical_accuracy: 0.3873
Epoch 154/1000
5/5 [==============================] - 0s 76ms/step - loss: 1.4465 - categorical_accuracy: 0.345

5/5 [==============================] - 0s 75ms/step - loss: 1.0181 - categorical_accuracy: 0.5211
Epoch 219/1000
5/5 [==============================] - 0s 78ms/step - loss: 1.0238 - categorical_accuracy: 0.5634
Epoch 220/1000
5/5 [==============================] - 0s 77ms/step - loss: 0.9248 - categorical_accuracy: 0.5845
Epoch 221/1000
5/5 [==============================] - 0s 83ms/step - loss: 0.8829 - categorical_accuracy: 0.6127
Epoch 222/1000
5/5 [==============================] - 0s 79ms/step - loss: 0.8508 - categorical_accuracy: 0.5986
Epoch 223/1000
5/5 [==============================] - 0s 79ms/step - loss: 0.8702 - categorical_accuracy: 0.6479
Epoch 224/1000
5/5 [==============================] - 0s 78ms/step - loss: 0.8154 - categorical_accuracy: 0.6761
Epoch 225/1000
5/5 [==============================] - 0s 76ms/step - loss: 0.7802 - categorical_accuracy: 0.6901
Epoch 226/1000
5/5 [==============================] - 0s 76ms/step - loss: 0.9589 - categorical_accuracy: 0.598

5/5 [==============================] - 0s 86ms/step - loss: 0.6591 - categorical_accuracy: 0.7606
Epoch 291/1000
5/5 [==============================] - 0s 76ms/step - loss: 0.5539 - categorical_accuracy: 0.8239
Epoch 292/1000
5/5 [==============================] - 0s 77ms/step - loss: 0.4729 - categorical_accuracy: 0.8451
Epoch 293/1000
5/5 [==============================] - 0s 86ms/step - loss: 0.4608 - categorical_accuracy: 0.8451
Epoch 294/1000
5/5 [==============================] - 0s 77ms/step - loss: 0.4283 - categorical_accuracy: 0.8732
Epoch 295/1000
5/5 [==============================] - 0s 80ms/step - loss: 0.3680 - categorical_accuracy: 0.8732
Epoch 296/1000
5/5 [==============================] - 0s 83ms/step - loss: 0.3037 - categorical_accuracy: 0.9296
Epoch 297/1000
5/5 [==============================] - 0s 79ms/step - loss: 0.3037 - categorical_accuracy: 0.9085
Epoch 298/1000
5/5 [==============================] - 0s 79ms/step - loss: 0.3493 - categorical_accuracy: 0.901

...

5/5 [==============================] - 0s 85ms/step - loss: 1.0104 - categorical_accuracy: 0.6408
Epoch 651/1000
5/5 [==============================] - 0s 89ms/step - loss: 1.1160 - categorical_accuracy: 0.6761
Epoch 652/1000
5/5 [==============================] - 0s 83ms/step - loss: 1.4732 - categorical_accuracy: 0.2465
Epoch 653/1000
5/5 [==============================] - 0s 85ms/step - loss: 1.1330 - categorical_accuracy: 0.4930
Epoch 654/1000
5/5 [==============================] - 0s 82ms/step - loss: 0.9511 - categorical_accuracy: 0.5915
Epoch 655/1000
5/5 [==============================] - 0s 87ms/step - loss: 0.8935 - categorical_accuracy: 0.6901
Epoch 656/1000
5/5 [==============================] - 0s 89ms/step - loss: 0.8386 - categorical_accuracy: 0.7606
Epoch 657/1000
5/5 [==============================] - 0s 88ms/step - loss: 0.7648 - categorical_accuracy: 0.7676
Epoch 658/1000
5/5 [==============================] - 0s 88ms/step - loss: 0.6632 - categorical_accuracy: 0.809

5/5 [==============================] - 0s 85ms/step - loss: 0.6988 - categorical_accuracy: 0.7676
Epoch 723/1000
5/5 [==============================] - 0s 83ms/step - loss: 0.6339 - categorical_accuracy: 0.8028
Epoch 724/1000
5/5 [==============================] - 0s 84ms/step - loss: 0.5582 - categorical_accuracy: 0.8451
Epoch 725/1000
5/5 [==============================] - 0s 87ms/step - loss: 0.5283 - categorical_accuracy: 0.8310
Epoch 726/1000
5/5 [==============================] - 0s 87ms/step - loss: 0.5168 - categorical_accuracy: 0.8310
Epoch 727/1000
5/5 [==============================] - 0s 86ms/step - loss: 0.4644 - categorical_accuracy: 0.8310
Epoch 728/1000
5/5 [==============================] - 0s 79ms/step - loss: 0.4380 - categorical_accuracy: 0.8521
Epoch 729/1000
5/5 [==============================] - 0s 82ms/step - loss: 0.4472 - categorical_accuracy: 0.8380
Epoch 730/1000
5/5 [==============================] - 0s 82ms/step - loss: 0.3926 - categorical_accuracy: 0.852

5/5 [==============================] - 0s 84ms/step - loss: 0.2932 - categorical_accuracy: 0.9014
Epoch 795/1000
5/5 [==============================] - 0s 82ms/step - loss: 0.2997 - categorical_accuracy: 0.8873
Epoch 796/1000
5/5 [==============================] - 0s 83ms/step - loss: 0.2767 - categorical_accuracy: 0.9155
Epoch 797/1000
5/5 [==============================] - 0s 85ms/step - loss: 0.2710 - categorical_accuracy: 0.9225
Epoch 798/1000
5/5 [==============================] - 0s 86ms/step - loss: 0.2850 - categorical_accuracy: 0.8944
Epoch 799/1000
5/5 [==============================] - 0s 87ms/step - loss: 0.4134 - categorical_accuracy: 0.8380
Epoch 800/1000
5/5 [==============================] - 0s 81ms/step - loss: 1.0092 - categorical_accuracy: 0.6620
Epoch 801/1000
5/5 [==============================] - 0s 80ms/step - loss: 0.6570 - categorical_accuracy: 0.8028
Epoch 802/1000
5/5 [==============================] - 0s 82ms/step - loss: 0.7234 - categorical_accuracy: 0.711

5/5 [==============================] - 1s 100ms/step - loss: 0.4095 - categorical_accuracy: 0.8732
Epoch 867/1000
5/5 [==============================] - 1s 101ms/step - loss: 0.4163 - categorical_accuracy: 0.8451
Epoch 868/1000
5/5 [==============================] - 1s 100ms/step - loss: 0.3369 - categorical_accuracy: 0.9085
Epoch 869/1000
5/5 [==============================] - 1s 99ms/step - loss: 0.3202 - categorical_accuracy: 0.9296
Epoch 870/1000
5/5 [==============================] - 1s 101ms/step - loss: 0.2881 - categorical_accuracy: 0.9437
Epoch 871/1000
5/5 [==============================] - 1s 104ms/step - loss: 0.2876 - categorical_accuracy: 0.9155
Epoch 872/1000
5/5 [==============================] - 1s 100ms/step - loss: 0.2979 - categorical_accuracy: 0.9014
Epoch 873/1000
5/5 [==============================] - 1s 101ms/step - loss: 0.2515 - categorical_accuracy: 0.9296
Epoch 874/1000
5/5 [==============================] - 1s 104ms/step - loss: 0.2471 - categorical_accurac

5/5 [==============================] - 1s 102ms/step - loss: 0.1969 - categorical_accuracy: 0.9507
Epoch 939/1000
5/5 [==============================] - 1s 104ms/step - loss: 0.2689 - categorical_accuracy: 0.9225
Epoch 940/1000
5/5 [==============================] - 1s 102ms/step - loss: 0.2473 - categorical_accuracy: 0.9296
Epoch 941/1000
5/5 [==============================] - 1s 104ms/step - loss: 0.1855 - categorical_accuracy: 0.9577
Epoch 942/1000
5/5 [==============================] - 1s 101ms/step - loss: 0.1808 - categorical_accuracy: 0.9507
Epoch 943/1000
5/5 [==============================] - 1s 101ms/step - loss: 0.2014 - categorical_accuracy: 0.9366
Epoch 944/1000
5/5 [==============================] - 1s 110ms/step - loss: 0.2149 - categorical_accuracy: 0.9296
Epoch 945/1000
5/5 [==============================] - 1s 105ms/step - loss: 0.1842 - categorical_accuracy: 0.9507
Epoch 946/1000
5/5 [==============================] - 1s 103ms/step - loss: 0.2025 - categorical_accura

In [38]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 5)                 165       
                                                                 
Total params: 596741 (2.28 MB)
Trainable params: 596741 

In [39]:
res = model.predict(x_test)

1/1 [==============================] - 1s 645ms/step


In [40]:
actions[np.argmax(res[6])]

'hello'

In [41]:
model.save('action.keras')

In [42]:
model.load_weights('action.keras')

In [43]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [44]:
yhat = model.predict(x_train)

5/5 [==============================] - 0s 28ms/step


In [45]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [46]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[108,   5],
        [  2,  27]],

       [[113,   2],
        [  2,  25]],

       [[113,   1],
        [  3,  25]],

       [[109,   4],
        [  3,  26]],

       [[113,   0],
        [  2,  27]]], dtype=int64)

In [47]:
accuracy_score(ytrue, yhat)

0.9154929577464789

In [48]:
accuracy_score(ytrue, yhat)

0.9154929577464789

In [49]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [50]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

TypeError: only length-1 arrays can be converted to Python scalars

<Figure size 1800x1800 with 0 Axes>

In [ ]:
sequence.reverse()

In [ ]:
len(sequence)

In [ ]:
sequence.append('def')

In [ ]:
sequence.reverse()

In [ ]:
sequence[-30:]

In [51]:
sequence = []
sentence = []
threshold = 0.8

# Load your trained model here
# model = load_your_model_here()

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)
        draw_styled_landmarks(image, results)

        keypoints = extract_keypoints(results)
        sequence.insert(0, keypoints)
        sequence = sequence[:30]

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])

            if np.any(res > threshold):
                if len(sentence) > 0:
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

        if len(sentence) > 5:
            sentence = sentence[-5:]

        cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        cv2.imshow('OpenCV Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 27ms/step
hello
1/1 [==============================] - 0s 21ms/step
hello
1/1 [==============================] - 0s 20ms/step
hello
1/1 [==============================] - 0s 19ms/step
hello
1/1 [==============================] - 0s 25ms/step
hello
1/1 [==============================] - 0s 20ms/step
hello
1/1 [==============================] - 0s 23ms/step
hello
1/1 [==============================] - 0s 19ms/step
hello
1/1 [==============================] - 0s 24ms/step
hello
1/1 [==============================] - 0s 24ms/step
hello
1/1 [==============================] - 0s 23ms/step
hello
1/1 [==============================] - 0s 20ms/step
hello
1/1 [==============================] - 0s 19ms/step
hello
1/1 [==============================] - 0s 21ms/step
hello
1/1 [==============================] - 0s 23ms/step
hello
1/1 [==============================] - 0s 22ms/step
hello
1/1 [==============================] - 0s 18ms/step
hello
1/1 [=========

1/1 [==============================] - 0s 25ms/step
hello
1/1 [==============================] - 0s 22ms/step
hello
1/1 [==============================] - 0s 24ms/step
hello
1/1 [==============================] - 0s 19ms/step
hello
1/1 [==============================] - 0s 20ms/step
hello
1/1 [==============================] - 0s 18ms/step
hello
1/1 [==============================] - 0s 22ms/step
hello
1/1 [==============================] - 0s 22ms/step
hello
1/1 [==============================] - 0s 21ms/step
hello
1/1 [==============================] - 0s 17ms/step
hello
1/1 [==============================] - 0s 21ms/step
thanks
1/1 [==============================] - 0s 22ms/step
thanks
1/1 [==============================] - 0s 24ms/step
hello
1/1 [==============================] - 0s 24ms/step
hello
1/1 [==============================] - 0s 23ms/step
hello
1/1 [==============================] - 0s 24ms/step
hello
1/1 [==============================] - 0s 28ms/step
hello
1/1 [=======

KeyboardInterrupt: 

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
res[np.argmax(res)] > threshold

In [ ]:
(num_sequences,30,1662)

In [ ]:
model.predict(np.expand_dims(X_test[0], axis=0))